In [175]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 7000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
eval_iters = 200
n_emb = 384

n_head = 6
n_layer = 6
dropout = 0.2

# ------------

# torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
with open('harrypotter.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [176]:
torch.manual_seed(1337)
B,T,C = 1,3,4 # batch size, sequence length, embedding size
# x = torch.randn(B,T,C)
x = [[[1,2,3,4],[5,6,7,8],[9,10,11,12]]]
x = torch.tensor(x, dtype=torch.float32)

# model definition
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
v = value(x)


tril = torch.tril(torch.ones(T, T)) 
wei = torch.matmul(q, k.transpose(-2,-1)) / (head_size ** 0.5)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
# print(wei.shape)

out = torch.matmul(wei, v)

In [177]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)

        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # A buffer is used because it is not a parameter of the model, but we want it on the GPU

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        wei = torch.matmul(q, k.transpose(-2,-1)) / (C ** 0.5)
        wei = wei.masked_fill(self.tril == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        out = torch.matmul(wei, v)
        return out

class MutliHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.proj = nn.Linear(head_size*n_heads, n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        out = torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd), # Projection Layer back to n_embd
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MutliHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

#Creating the Batch Normalization Layer 
class BatchNorm1d: 
    def __init__(self,dim,eps=1e-5,momentum=.1):
        self.eps = eps
        self.momentum = momentum
        self.dim = dim
        # self.weight = torch.ones(dim) #Gain
        # self.bias = torch.zeros(dim) #Bias
        self.weight = torch.ones(dim) #Gain
        self.bias = torch.zeros(dim) #Bias
    

        self.params = [self.weight,self.bias]

        self.training = True

    def __call__(self,x):
        #Calculate Forward Pass

        #Use batch mean
        dim = 1
        xmean = x.mean(dim=dim, keepdim=True)
        xvar = x.var(dim=dim, keepdim=True)
        

        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) #Make it a gaussian (mean 0, std 1)
        self.out = self.weight * xhat + self.bias #Scale and shiftA

        

        return self.out

    def parameters(self):
        return self.params 
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        # self.lm_head = nn.Linear(n_emb, vocab_size)
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(n_layer)] + [nn.LayerNorm(n_emb)])
        self.lm_head = nn.Linear(n_emb, vocab_size) # converge back to vocab size so that we can predict the next token 

    def forward(self, idx, targets=None):

        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,n_emb)
        logits = self.lm_head(x) # (B,T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            #Crop idx to block_size
            idx_cond = idx[:, -block_size:]

            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [178]:
model = BigramLanguageModel()
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [179]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.3569, val loss 4.3604
step 500: train loss 2.0065, val loss 2.0951
step 1000: train loss 1.6105, val loss 1.7863
step 1500: train loss 1.4404, val loss 1.6360
step 2000: train loss 1.3483, val loss 1.5717
step 2500: train loss 1.2804, val loss 1.5277
step 3000: train loss 1.2318, val loss 1.5033
step 3500: train loss 1.1898, val loss 1.4870
step 4000: train loss 1.1529, val loss 1.4832
step 4500: train loss 1.1187, val loss 1.4784
step 5000: train loss 1.0852, val loss 1.4832
step 5500: train loss 1.0504, val loss 1.5027
step 6000: train loss 1.0191, val loss 1.4962
step 6500: train loss 0.9856, val loss 1.5165


In [180]:
# generate from the model
context = torch.zeros((1, 256), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=5000)[0].tolist())[256:])

LLA.EDWARCLAS:
CLAUFIS:
All, my wounds:
Go: bo by and halloopp him back.

His nobortomages:
How now, I warrandly, will, madam: if we chare him no title,
Beat him turning, as we should revenged on one
Thus chase would trust have lessly.

ROMEO:
O, for my mindy! O my sovereign, my lord!
Abo's father: I would I were here not spirit
That; when I wis. If it condit, sirrah,
If you do deserve your friends, here with our
To redirest your complaining is full of sport:
It is too commandment, too much nettless
To maintard, you hither will chowarment.
We may be Dester? No, n aught and Richard

HORTENSIO:
A grave, I have mine, people so much unle:
Unto the despairing as shall havews of him
As I to ream coming: therefore, the them
Down in the common our formasties of you: your eyes,
You must we do fear, and, I obear,--

Nurse:
Good Catesby, and, dispatch-enobled,
Would but negliciously?

CATESBY:
Fave was an't; yes from the best sense this bloth,
And beg on your tears of its,--
Let them be by wind, 

In [181]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters
